In [ ]:
def train_mapper(optimizer, generated_w):
  
    optimizer.zero_grad()
    prediction = discriminator(generated_w).view(-1)
    # Calculate error and backpropagate
    error = calc_Dw_loss(prediction, 1, "cuda", generated_w, config['R1Param'], False)
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error, prediction

In [ ]:
epochs = 4
for epoch in range(epochs):
  for idx, data in enumerate(train_loader):

    id_images, attr_images, ws = data

    torch.cuda.empty_cache()
    id_images = id_images[0].cuda()
    attr_images = attr_images[0].cuda()
    ws = ws.cuda()

    if idx % config['IdDiffersAttrTrainRatio'] == 0:
      different_attr_images = torch.empty_like(attr_images, device='cuda')
      different_attr_images[0] = attr_images[7]
      different_attr_images[1:] = attr_images[:7]
      attr_images = different_attr_images
    
    with torch.no_grad():
      id_vec = E_id(id_images)

    attr_vec = E_att(attr_images)
    id_vec = torch.squeeze(id_vec)
    attr_vec = torch.squeeze(attr_vec)
    encoded_vec = torch.cat((id_vec,attr_vec), dim=1)

    fake_data = mlp(encoded_vec)
    error_real, prediction_real, error_fake, prediction_fake = train_discriminator(optimizerD, ws, fake_data)
    print(f"\n error_real: {error_real}, error_fake: {error_fake} \n prediction_real: {torch.mean(prediction_real)}, prediction_fake: {torch.mean(prediction_fake)}")
    g_error, g_pred = train_mapper(optimizerMLP, fake_data)
    print(f"\n g_error: {g_error}, g_pred: {torch.mean(g_pred)}")

    MLP_losses.append(g_error)
    D_losses.append((error_real + error_fake) /2)

    if idx % 5 == 0:
        with torch.no_grad():
          plot_w_image(mlp(test_vec)[0])